In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
df = pd.read_csv('C:/Users/butle/hp/Capstone_Two/data/interim/combined_health_ejscreen.csv')

In [14]:
df.columns.values

array(['Unnamed: 0', 'StateAbbr', 'CountyName', 'LocationName',
       'Category', 'Measure', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Low_Confidence_Limit', 'High_Confidence_Limit',
       'TotalPopulation', 'LocationID', 'CategoryID', 'MeasureId',
       'Short_Question_Text', 'MINORPCT', 'LOWINCPCT', 'LESSHSPCT',
       'LINGISOPCT', 'UNDER5PCT', 'OVER64PCT', 'PRE1960PCT', 'UNEMPPCT',
       'VULEOPCT', 'DSLPM', 'CANCER', 'RESP', 'PTRAF', 'PWDIS', 'PNPL',
       'PRMP', 'PTSDF', 'OZONE', 'PM25', 'UST', 'D_LDPNT_2', 'D_DSLPM_2',
       'D_CANCR_2', 'D_RESP_2', 'D_PTRAF_2', 'D_PWDIS_2', 'D_PNPL_2',
       'D_PRMP_2', 'D_PTSDF_2', 'D_OZONE_2', 'D_PM25_2', 'D_UST_2',
       'P_MINORPCT', 'P_LWINCPCT', 'P_LESHSPCT', 'P_LNGISPCT',
       'P_UNDR5PCT', 'P_OVR64PCT', 'P_LDPNT', 'P_UNEMPPCT', 'P_VULEOPCT',
       'P_DSLPM', 'P_CANCR', 'P_RESP', 'P_PTRAF', 'P_PWDIS', 'P_PNPL',
       'P_PRMP', 'P_PTSDF', 'P_OZONE', 'P_PM25', 'P_UST', 'P_LDPNT_D2',
       'P_DSLPM_D2', 'P_CA

In [15]:
df.head()

,Unnamed: 0,StateAbbr,CountyName,LocationName,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,Low_Confidence_Limit,...,P_DSLPM_D2,P_CANCR_D2,P_RESP_D2,P_PTRAF_D2,P_PWDIS_D2,P_PNPL_D2,P_PRMP_D2,P_PTSDF_D2,P_OZONE_D2,P_PM25_D2
0,0,AL,Butler,1013953200,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.3,10.0,...,61.875069,61.980609,62.200554,54.821884,51.660665,61.921607,60.029640,60.672022,62.606094,62.466759
1,1,AL,Butler,1013953200,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Crude prevalence,6.8,6.6,...,61.875069,61.980609,62.200554,54.821884,51.660665,61.921607,60.029640,60.672022,62.606094,62.466759
2,2,AL,Cleburne,1029959500,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,9.6,...,49.943384,46.943384,46.943384,58.118593,38.840871,42.180570,54.943384,54.530988,45.943384,46.649581
3,3,AL,Cleburne,1029959500,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Crude prevalence,6.8,6.6,...,49.943384,46.943384,46.943384,58.118593,38.840871,42.180570,54.943384,54.530988,45.943384,46.649581
4,4,AL,Dallas,1047956600,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,12.3,11.6,...,78.048556,81.485140,80.071997,83.987861,70.077438,73.602763,82.245710,63.746337,78.959397,79.461699
